Raster CLI and API sidebyside

In [1]:
import os
os.chdir(r"D:\1. PROJECT\satellite_etl\satellite\reference image")
os.getcwd()

'D:\\1. PROJECT\\satellite_etl\\satellite\\reference image'

In [2]:
!gdalinfo oldimage.tif
!gdalinfo -stats oldimage.tif

Driver: GTiff/GeoTIFF
Files: oldimage.tif
       oldimage.tif.aux.xml
Size is 2550, 2809
Coordinate System is:
PROJCRS["UTM_M_PRS92",
    BASEGEOGCRS["PRS92",
        DATUM["Philippine Reference System 1992",
            ELLIPSOID["Clarke 1866",6378206.4,294.978698213898,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4683]],
    CONVERSION["Transverse Mercator",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",123,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",1],
    

In [5]:
from osgeo import gdal

ds = gdal.Open("oldimage.tif")
print(ds.RasterXSize, ds.RasterYSize)
print(ds.RasterCount)
print(ds.GetProjection())

2550 2809
5
PROJCS["UTM_M_PRS92",GEOGCS["PRS92",DATUM["Philippine_Reference_System_1992",SPHEROID["Clarke 1866",6378206.4,294.978698213898,AUTHORITY["EPSG","7008"]],AUTHORITY["EPSG","6683"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4683"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",123],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [16]:
!gdalwarp -t_srs EPSG:4326 -tr 30 30 -te 525162.384 1426066.097 526437.384 1427470.472 -r bilinear oldimage.tif aligned.tif

Using band 5 of source image as alpha.
Creating output file that is 43P x 47L.
Processing oldimage.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PROJ: cart: Invalid latitude
ERROR 1: PRO

In [13]:
!gdalinfo ref.tif

Driver: GTiff/GeoTIFF
Files: ref.tif
Size is 2040, 2247
Coordinate System is:
PROJCRS["UTM_M_PRS92",
    BASEGEOGCRS["PRS92",
        DATUM["Philippine Reference System 1992",
            ELLIPSOID["Clarke 1866",6378206.4,294.978698213898,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4683]],
    CONVERSION["Transverse Mercator",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",123,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",1],
            ID["EPSG",8806]],
       

Getting RGB and NIR in the satellite data

In [ ]:
!gdalinfo oldimage.tif

In [18]:
!gdal_translate -b 1 -b 2 -b 3 oldimage.tif rgb.tif

Input file size is 2550, 2809
0...10...20...30...40...50...60...70...80...90...100 - done.


In [20]:
!gdal_translate -b 4 oldimage.tif nir.tif

Input file size is 2550, 2809
0...10...20...30...40...50...60...70...80...90...100 - done.


Using osgeo_gdal

In [21]:
from osgeo import gdal

infile = "oldimage.tif"
ds = gdal.Open(infile)

driver = gdal.GetDriverByName("GTiff")
rgb_ds = driver.Create("rgb_python.tif", ds.RasterXSize, ds.RasterYSize, 3, gdal.GDT_UInt16)
rgb_ds.SetProjection(ds.GetProjection())
rgb_ds.SetGeoTransform(ds.GetGeoTransform())
rgb_ds.GetRasterBand(1).WriteArray(ds.GetRasterBand(1).ReadAsArray())  # Red
rgb_ds.GetRasterBand(2).WriteArray(ds.GetRasterBand(2).ReadAsArray())  # Green
rgb_ds.GetRasterBand(3).WriteArray(ds.GetRasterBand(3).ReadAsArray())  # Blue
rgb_ds = None  # close

# --- Save NIR ---
nir_ds = driver.Create("nir_python.tif", ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_UInt16)
nir_ds.SetProjection(ds.GetProjection())
nir_ds.SetGeoTransform(ds.GetGeoTransform())
nir_ds.GetRasterBand(1).WriteArray(ds.GetRasterBand(4).ReadAsArray())  # NIR
nir_ds = None  # close


Using Rasterio

In [ ]:
import rasterio
import numpy as np

infile = "oldimage.tif"

with rasterio.open(infile) as src:
    profile = src.profile

    # Read bands
    red   = src.read(1)
    green = src.read(2)
    blue  = src.read(3)
    nir   = src.read(4)

    # --- Save RGB ---
    rgb_profile = profile.copy()
    rgb_profile.update(count=3)  # 3-band output
    with rasterio.open("rgb.tif", "w", **rgb_profile) as dst:
        dst.write(red, 1)
        dst.write(green, 2)
        dst.write(blue, 3)

    # --- Save NIR ---
    nir_profile = profile.copy()
    nir_profile.update(count=1)  # single-band output
    with rasterio.open("nir.tif", "w", **nir_profile) as dst:
        dst.write(nir, 1)
